In [1]:
import os 
# os.environ["HF_HOME"] = "./transformers_cache"
# os.environ["TRANSFORMERS_CACHE"] = "./transformers_cache"


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import types
import matplotlib.pyplot as plt
from collections import defaultdict

# Load model
model_name = "ibm-granite/granite-3.1-3b-a800m-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Get actual number of experts (confirmed 40 for Granite-3B)
num_experts = 40
print(f"Model has {num_experts} experts per layer")

/home2/abraham.elenjical/miniconda3/envs/res_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home2/abraham.elenjical/miniconda3/envs/res_env/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install

Model has 40 experts per layer


In [ ]:
print(model)

GraniteMoeForCausalLM(
  (model): GraniteMoeModel(
    (embed_tokens): Embedding(49155, 1536, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x GraniteMoeDecoderLayer(
        (self_attn): GraniteMoeSdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (k_proj): Linear(in_features=1536, out_features=512, bias=False)
          (v_proj): Linear(in_features=1536, out_features=512, bias=False)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (block_sparse_moe): GraniteMoeMoE(
          (activation): SiLU()
          (input_linear): GraniteMoeParallelExperts()
          (output_linear): GraniteMoeParallelExperts()
          (router): GraniteMoeTopKGating(
            (layer): Linear(in_features=1536, out_features=40, bias=False)
          )
        )
        (input_layernorm): GraniteMoeRMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): GraniteMoeRMSNorm((1536,), eps=1e-06)
   

In [5]:
def print_module_tree(module, indent=0):
    print(" " * indent + f"{module.__class__.__name__}")
    for child in module.children():
        print_module_tree(child, indent + 2)

print_module_tree(model)


GraniteMoeForCausalLM
  GraniteMoeModel
    Embedding
    ModuleList
      GraniteMoeDecoderLayer
        GraniteMoeSdpaAttention
          Linear
          Linear
          Linear
          Linear
        GraniteMoeMoE
          SiLU
          GraniteMoeParallelExperts
          GraniteMoeParallelExperts
          GraniteMoeTopKGating
            Linear
        GraniteMoeRMSNorm
        GraniteMoeRMSNorm
      GraniteMoeDecoderLayer
        GraniteMoeSdpaAttention
          Linear
          Linear
          Linear
          Linear
        GraniteMoeMoE
          SiLU
          GraniteMoeParallelExperts
          GraniteMoeParallelExperts
          GraniteMoeTopKGating
            Linear
        GraniteMoeRMSNorm
        GraniteMoeRMSNorm
      GraniteMoeDecoderLayer
        GraniteMoeSdpaAttention
          Linear
          Linear
          Linear
          Linear
        GraniteMoeMoE
          SiLU
          GraniteMoeParallelExperts
          GraniteMoeParallelExperts
          Gra